# SolarWinds Report to PDF

Please contact Tom Ling (Data Analyst, Ekco/Frontier Technology) if updating this file.  
*v1.1.0, last updated: 2024-07-31*

This Notebook generates PDFs from SolarWinds Orion report URLs.

## Instructions
1. Run **Prepare Notebook** section.
2. Add username and password to Configuration settings, credentials section (for SolarWinds reports). 
3. Run required cell(s) in the **Data processing** section. 

Files are outputted to the working directory. 

## Prepare Notebook

In [1]:
# Standard library imports
import sys
import os

In [2]:
# Create PDF from webpage in Python
# https://medium.com/@nikitatonkoshkur25/create-pdf-from-webpage-in-python-1e9603d6a430

import base64
import json
import logging
import time
from io import BytesIO
from typing import List

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [8]:
# Define functions that handle login to SolarWinds Orion reporting
class PdfGenerator:
    """
     Simple use case:
        pdf_file = PdfGenerator(['https://google.com']).main()
        with open('new_pdf.pdf', "wb") as outfile:
            outfile.write(pdf_file[0].getbuffer())
    """
    driver = None
    # https://chromedevtools.github.io/devtools-protocol/tot/Page#method-printToPDF
    print_options = {
        'landscape': False,
        'displayHeaderFooter': False,
        'printBackground': True,
        'preferCSSPageSize': True,
        'paperWidth': 8.5,
        'paperHeight': 11,
    }

    def __init__(self, urls: List[str]):
        self.urls = urls

    def _get_pdf_from_url(self, url, *args, **kwargs):
        self.driver.get(url)
        
        if use_solarwinds:
            time.sleep(sw_sleep) # allow the page to load, increase if needed
        
            # SolarWinds reports have unwanted HTML elements if accessed via Selenium. 
            # These need to be removed before PDFing. 
            self.driver.execute_script(
            """
            var l = document.getElementsByClassName("xui sw-dashboard__editing-buttons")[0];
            l.parentNode.removeChild(l);
            """
            )
            self.driver.execute_script(
            """
            var l = document.getElementsByClassName("sw-mainnav-branding")[0];
            l.parentNode.removeChild(l);
            """
            )
            self.driver.execute_script(
            """
            var l = document.getElementsByClassName(" sw-btn-secondary sw-btn")[0];
            l.parentNode.removeChild(l);
            """
            )
            self.driver.execute_script(
            """
            var l = document.getElementsByClassName("sw-hdr-links sw-hdr-links-nested")[0];
            l.parentNode.removeChild(l);
            """
            )
            self.driver.execute_script(
            """
            var l = document.getElementsByClassName("sw-pg-rpthide")[0];
            l.parentNode.removeChild(l);
            """
            )
        else:
            time.sleep(0.5)
        
        print_options = self.print_options.copy()
        result = self._send_devtools(self.driver, "Page.printToPDF", print_options)
        return base64.b64decode(result['data'])

    @staticmethod
    def _send_devtools(driver, cmd, params):
        """
        Works only with chromedriver.
        Method uses chromedriver's api to pass various commands to it.
        """
        resource = "/session/%s/chromium/send_command_and_get_result" % driver.session_id
        url = driver.command_executor._url + resource
        body = json.dumps({'cmd': cmd, 'params': params})
        response = driver.command_executor._request('POST', url, body)
        return response.get('value')

    def _generate_pdfs(self):
        pdf_files = []

        for url in self.urls:
            result = self._get_pdf_from_url(url)
            file = BytesIO()
            file.write(result)
            pdf_files.append(file)

        return pdf_files

    def main(self) -> List[BytesIO]:
        webdriver_options = ChromeOptions()
        # Makes Chrome run without opening a window.
        webdriver_options.add_argument('--headless')
        webdriver_options.add_argument('--disable-gpu')

        try:
            self.driver = webdriver.Chrome(
                service=ChromeService(ChromeDriverManager().install()),
                options=webdriver_options
            )
            
            if use_solarwinds: 
                # Login to SolarWinds using a user account. 

                self.driver.get(sw_login_url) 
                time.sleep(2)

                username = self.driver.find_element("id", "ctl00_BodyContent_Username")
                password = self.driver.find_element("id", "ctl00_BodyContent_Password")

                username.send_keys(sw_username)
                password.send_keys(sw_password)

                login_attempt = self.driver.find_element("id", "ctl00_BodyContent_LoginButton")
                login_attempt.submit()
                time.sleep(2)
               
            result = self._generate_pdfs()
        finally:
            self.driver.close()

        return result

In [14]:
# Configuration settings that trigger specific behaviour if using SolarWinds. 

# Set to false if using this workbook for other sites. 
use_solarwinds = True

# Set time delay in seconds, for SolarWinds reports (default: 12s)
sw_sleep = 12

# Credentials. 
sw_login_url = "url.com"
sw_username = "username"
sw_password = ""

# Inputs

In [44]:
# Test using Google. 
use_solarwinds = False
site_url = "https://google.com"
pdf_filename = "google.pdf"

# Data processing

In [45]:
# Get PDF of site.
pdf_file = PdfGenerator([site_url]).main()
with open(pdf_filename, "wb") as outfile:
    outfile.write(pdf_file[0].getbuffer())
print(pdf_filename + " saved.")

google.pdf saved.


In [ ]:
# Multiple reports (here, for FFDB Monthly Infrastructure Reports). 
# Supply file name and report url as a list of list of str.
print("Start of loop.")
for url in [['Report Name 1', 'reporturl.aspx?ReportID=464'], 
            ]: 
    report_url = url[1]
    report_name = url[0] + ".pdf"

    pdf_file = PdfGenerator([report_url]).main()
    with open(report_name, "wb") as outfile:
        outfile.write(pdf_file[0].getbuffer())
    print(report_name + " saved.")
print("End of loop.")

In [ ]:
# One report.
# Supply file name and report url as a list of list of str.
print("Start of loop.")
for url in [['Report Name 1', 'reporturl.aspx?ReportID=100']]: 
    report_url = url[1]
    report_name = url[0] + ".pdf"

    pdf_file = PdfGenerator([report_url]).main()
    with open(report_name, "wb") as outfile:
        outfile.write(pdf_file[0].getbuffer())
    print(report_name + " saved.")
print("End of loop.")

In [ ]:
# Multiple reports (here, for Monthly Review). 
# Supply file name and report url as a list of list of str.
print("Start of loop.")
for url in [['Report Name 1', 'reporturl.aspx?ReportID=100'],
           ['Report Name 2', 'reporturl.aspx?ReportID=101']]: 
    report_url = url[1]
    report_name = url[0] + ".pdf"

    pdf_file = PdfGenerator([report_url]).main()
    with open(report_name, "wb") as outfile:
        outfile.write(pdf_file[0].getbuffer())
    print(report_name + " saved.")
print("End of loop.")

In [ ]:
# Client Weekly Reports

# Increase time delay to 15s.
sw_sleep = 50

# Supply file name and report url as a list of list of str.
print("Start of loop.")
for url in [['Report Name 1', 'reporturl.aspx?ReportID=100'],
           ['Report Name 2', 'reporturl.aspx?ReportID=101']]: 
    report_url = url[1]
    report_name = url[0] + ".pdf"

    pdf_file = PdfGenerator([report_url]).main()
    with open(report_name, "wb") as outfile:
        outfile.write(pdf_file[0].getbuffer())
    print(report_name + " saved.")
print("End of loop.")

# Outputs

In [ ]:
# Files are put in the working directory. 